# Employee Review Demo

A demo of using [SurrealDB](https://surrealdb.com/) to query Kaggle's [Employee Review](https://www.kaggle.com/datasets/fiodarryzhykau/employee-review) dataset.

In [175]:
%pip -q install surrealdb pandas

Note: you may need to restart the kernel to use updated packages.


## Get started

### Run database in container

First, run SurrealDB as a container. I use [podman](https://podman.io/) because I like it better, but docker will work too.

The volume mount is optional, in case you want to export your database.

```{danger}
Passing credentials like this is insecure and is for testing and demo only!
```

```bash
podman run --rm -it -p 8000:8000 -v `pwd`/mydata:/mydata docker.io/surrealdb/surrealdb:latest start --auth --user root --pass testing 
```

You should see

> INFO surrealdb::net: Started web server on 0.0.0.0:8000

### Login to database

We will use the [Python SDK for SurrealDB](https://docs.surrealdb.com/docs/integration/sdks/python/).

Note that I believe Python is an awful language and you should avoid it in most production cases. But for this demo notebook, eh, I suppose I can hold my nose and do it.

SurrealDB has a plethora of SDKs to chose from, though they aren't as developed as one might hope.

In [176]:
from surrealdb import Surreal

db = Surreal("http://localhost:8000")
await db.connect()

# NOTE: This is an insecure way of handling credentials and should not be used in production.
await db.signin({"user": "root", "pass": "testing"})
await db.use("test", "test")

print(db.client_state)

ConnectionState.CONNECTED


## Preview data

Again, this dataset is from Kaggle's [Employee Review](https://www.kaggle.com/datasets/fiodarryzhykau/employee-review).

Let's check it out. We'll also grab a list of unique names because we will need those.

In [177]:
# Preview the CSV
import pandas as pd

# Load the CSV file
file_path = 'employee_review_mturk_dataset_test_v6_kaggle.csv'
data = pd.read_csv(file_path)

# Display the info about the dataframe
print(data.info())

# For each unique name in the CSV
unique_names = data['person_name'].unique()
print("Found", len(unique_names), "unique names")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 225 non-null    int64 
 1   person_name        225 non-null    object
 2   nine_box_category  225 non-null    object
 3   feedback           225 non-null    object
 4   updated            225 non-null    bool  
 5   reviewed           225 non-null    bool  
dtypes: bool(2), int64(1), object(3)
memory usage: 7.6+ KB
None
Found 75 unique names


## Create member table

The first thing we will do with our database is create table to put the members of our company in. Let's call it **member**.

We will use a [schemafull table](https://docs.surrealdb.com/docs/surrealql/statements/define/table#schemafull-tables)
because then the database will strictly enforce the definitions we add to the table for each record.

While schemaless tables offer advantages in flexibility, for something as critical as the cornerstone record of who an employee is, schemafull is *absolutely* the way to go.

The queries can get a tad unwieldy, so I like to preview the string before sending to the database.

In [178]:
member_tb = "member"
createMemberQuery = f"""
# Create primary table for holding member info
DEFINE TABLE {member_tb} SCHEMAFULL;

# Name
DEFINE FIELD name ON TABLE {member_tb} TYPE string;

# Employee ID as unique index
DEFINE FIELD employeeId ON TABLE {member_tb} TYPE string
    ASSERT string::is::numeric(employeeId) && string::len(employeeId) == 5;
DEFINE INDEX employeeIdIndex ON TABLE {member_tb} COLUMNS employeeId UNIQUE;
"""

print(createMemberQuery)


# Create primary table for holding member info
DEFINE TABLE member SCHEMAFULL;

# Name
DEFINE FIELD name ON TABLE member TYPE string;

# Employee ID as unique index
DEFINE FIELD employeeId ON TABLE member TYPE string
    ASSERT string::is::numeric(employeeId) && string::len(employeeId) == 5;
DEFINE INDEX employeeIdIndex ON TABLE member COLUMNS employeeId UNIQUE;



In [179]:
# If the command to create the member table looks good, go ahead and create it.
await db.query(createMemberQuery)
# Verify the table was created
await db.query("INFO FOR TABLE member;")

[{'result': {'events': {},
   'fields': {'employeeId': 'DEFINE FIELD employeeId ON member TYPE string ASSERT string::is::numeric(employeeId) AND string::len(employeeId) == 5 PERMISSIONS FULL',
    'name': 'DEFINE FIELD name ON member TYPE string PERMISSIONS FULL'},
   'indexes': {'employeeIdIndex': 'DEFINE INDEX employeeIdIndex ON member FIELDS employeeId UNIQUE'},
   'lives': {},
   'tables': {}},
  'status': 'OK',
  'time': '2.48673ms'}]

### employeeId

Let's randomly assign each employee an ID number. Recall that the schema definition above enforces that `employeeId` is

- **UNIQUE** amongst all `employeeIds`
- A string made up of 5 digits
- [Indexed](https://docs.surrealdb.com/docs/surrealql/statements/define/indexes) with `employeeIdIndex`.

The record creation here will fail if any of those criteria (as well as the other definitions) are not met.

In [180]:
from random import randint

# Roster of entries
records_added = 0
failed_records: list[dict[str, str]] = []

# Add the person to the member table
print("Adding members to table:", member_tb)
for name in unique_names:
    # Create a random employee id of 5 digits
    employee_id = ''.join(str(randint(0, 9)) for _ in range(5))
    try:
        new_record = await db.create(member_tb, {
        'name': name,
        'employeeId': employee_id,
        })
        records_added = records_added + 1
    except Exception as e:
        failed_records.append({name, e})

print("Added records:", records_added)
print("Errors adding:", len(failed_records))
print("Failed records:\n", failed_records)

Adding members to table: member
Added records: 75
Errors adding: 0
Failed records:
 []


In [181]:
# View some of the results
await db.query(f"SELECT * FROM {member_tb} LIMIT 5;")

[{'result': [{'employeeId': '57039',
    'id': 'member:0ofhpdiywbksd94y6i5r',
    'name': 'Jayden Rees'},
   {'employeeId': '11808',
    'id': 'member:123y4mxnx06abumur88d',
    'name': 'Braiden Santos'},
   {'employeeId': '31786',
    'id': 'member:1cnbys0j93u40jpbw0ou',
    'name': 'Daisy Pearce'},
   {'employeeId': '18361',
    'id': 'member:2q7ls7jps6hlf4pvhaqb',
    'name': 'Heidi Wallace'},
   {'employeeId': '36579',
    'id': 'member:2unvbh258ug0c96eza12',
    'name': 'Logan Ellis'}],
  'status': 'OK',
  'time': '65.176µs'}]

## Add performance feedback

Now that we have our employees loaded in to the database, let's add the performance feedback!

### Create feedback table

We'll start out by making another schemafull table: **feedback**. I advocate for schemafull here because I'm imagining a standard feedback form with fields that must be filled in.

Additionally, there are some advantages for queries if you standardize fields. We will see this in action below.

In [182]:
# Create a schemafull table for feedback
feedback_tb = "feedback"
createFeedbackQuery = f"""
DEFINE TABLE {feedback_tb} SCHEMAFULL;
DEFINE FIELD ratee ON TABLE {feedback_tb} TYPE record<member>;
DEFINE FIELD rater ON TABLE {feedback_tb} TYPE record<member>;
DEFINE FIELD date on TABLE {feedback_tb} TYPE datetime;
DEFINE FIELD body ON TABLE {feedback_tb} TYPE string;
# Create index for faster query
DEFINE INDEX memberIndex ON TABLE {feedback_tb} COLUMNS member;
"""

print(createFeedbackQuery)


DEFINE TABLE feedback SCHEMAFULL;
DEFINE FIELD ratee ON TABLE feedback TYPE record<member>;
DEFINE FIELD rater ON TABLE feedback TYPE record<member>;
DEFINE FIELD date on TABLE feedback TYPE datetime;
DEFINE FIELD body ON TABLE feedback TYPE string;
# Create index for faster query
DEFINE INDEX memberIndex ON TABLE feedback COLUMNS member;



In [183]:
# If the command to create the table looks good, go ahead and create it.
await db.query(createFeedbackQuery)
# Verify the table was created
print(await db.query(f"INFO FOR TABLE {feedback_tb};"))

[{'result': {'events': {}, 'fields': {'body': 'DEFINE FIELD body ON feedback TYPE string PERMISSIONS FULL', 'date': 'DEFINE FIELD date ON feedback TYPE datetime PERMISSIONS FULL', 'ratee': 'DEFINE FIELD ratee ON feedback TYPE record<member> PERMISSIONS FULL', 'rater': 'DEFINE FIELD rater ON feedback TYPE record<member> PERMISSIONS FULL'}, 'indexes': {'memberIndex': 'DEFINE INDEX memberIndex ON feedback FIELDS member'}, 'lives': {}, 'tables': {}}, 'status': 'OK', 'time': '128.745µs'}]


### Add Feedback

To add the feedback to our database we'll make a few assumptions. Specifically, we'll randomly select and assign a manager and a recent date.

Each addition will require both
1. Creation of a record in the **feedback** table.
2. Creation of an edge in the **gotFeedback** edge table.

If either one fails, they both should fail. In order to do this, we'll make the addition as a [transaction](https://docs.surrealdb.com/docs/surrealql/transactions/) that we will then [commit](https://docs.surrealdb.com/docs/surrealql/statements/commit).

In [184]:
# Assume managers are the last 5 names in the list
managers = unique_names[-5:]
print(managers)

['Jaxson Giles' 'Heidi Wallace' 'Zachary Doyle' 'Lauren Baker'
 'George Jones']


In [185]:
from random import choice

# Go through each feedback and add it to the database with a randomly selected manager
feedback_df = data[['person_name','feedback']]
# Roster of entries
records_added = 0
failed_records: list[dict[str, str]] = []

# Add feedback
for _, row in feedback_df.iterrows():
    # Randomly generate datetime for when feedback happened
    feedbackDate = str(randint(2019, 2023)) + "-" + str(randint(10,12)) + "-" + str(randint(10,28))
    feedbackDate = feedbackDate + "T00:00:00Z"
    try:
        # Randomly select rater and get their member id
        rater = choice(managers)
        raterId = await db.query("SELECT id FROM member WHERE name = $rater;", {'rater':rater})
        raterId = raterId[0]["result"][0]["id"]
        # Get id of member. Doing this with employeeId would be better, but dataset limits
        memberId = await db.query(f"""SELECT id FROM member WHERE name = "{row.person_name}";""")
        memberId = memberId[0]["result"][0]["id"]
        query = f"""BEGIN TRANSACTION;
                  -- Create feedback record
                  LET $thisFeedbackId = (CREATE feedback CONTENT {{
                    'ratee': {memberId},
                    'rater': {raterId},
                    'date': "{feedbackDate}",
                    'body': "{row.feedback.replace('"', "'")}",   
                  }});
                  -- Relate feedback record to member
                  RELATE {memberId}->gotFeedback->$thisFeedbackId;
                  -- Transactions both work or both fail
                  COMMIT TRANSACTION;
                  """
        await db.query(query)
        records_added = records_added + 1
    except Exception as e:
        failed_records.append({name, e})

print("Added records:", records_added)
print("Errors adding:", len(failed_records))
print("Failed records:\n", failed_records)

    

Added records: 225
Errors adding: 0
Failed records:
 []


### Query Feedback Records

In this example we'll compare a traditional table query with a graph query. The objective is to get the text and rater for each and every feedback an employee has ever received.

Remember that our database currently has:
- A `member` table that holds basic information about employees.
- A `feedback` table that holds a record for every feedback.
- A `gotFeedback` [edge table](https://docs.surrealdb.com/docs/surrealql/statements/relate) that points from an employee to their feedback.

Furthermore, our `member` table has an INDEX on the `employeeId` column.

We will use **Valeria Crane** as our example employee. First, let's get her employee ID.
This would be known in a production environment - probably even printed on the back of a badge - so
it makes sense to somewhat artificially get it here.

In [186]:
# Get information about employee named Valeria Crane
crane = await db.query("SELECT * FROM member WHERE name = 'Valeria Crane'")
crane_employeeId = crane[0]['result'][0]['employeeId']
print("Crane's employeeId:", crane_employeeId)

Crane's employeeId: 65810


Run the table query first. This matches the approach taken by a traditional relational database.
Notice the `time` output at the bottom (about 3 milliseconds on my computer).

In [187]:
# Give me the rater name and the body of the feedback for every feedback the employee has gotten
crane_feedback_table = f"""SELECT rater.name AS rater, body FROM feedback WHERE ratee.employeeId = '{crane_employeeId}'"""

await db.query(crane_feedback_table)

[{'result': [{'body': 'Valeria Crane is a hazard in our team. She has shown no signs of reliability. She is difficult to work with and does not seem motivated. Valeria needs a lot of guidance to complete tasks.',
    'rater': 'Heidi Wallace'},
   {'body': 'To simply but, Valeria is currently ranked last in the customer service leaderboard, having missed some basic but important sales target and slacking off at work. She is constantly on her phone to the point where we had to bar the use of phones within the office, causing inconvenience to genuine team members. I am afraid, unless she improves, she may be counting her last few weeks in this company.',
    'rater': 'Jaxson Giles'},
   {'body': 'Valeria is considered a risk. She has not been performing up to required standards. We have tried working with her to improve and there has not been any improvement. More testing will be required, but I would not recommend for personal advancement until noticeable and consistent improvement from 

Now get the same information with a graph query. The output format will look slightly different, but it's still JSON.
Check out the `time` though! On my computer it's about 500 microseconds, or **6x as fast!**

In [188]:
# Get all of Crane's feedback using a graph query
crane_feedback_graph = f"""SELECT ->gotFeedback.out.rater.name AS rater, ->gotFeedback.out.body AS body
                            FROM (SELECT id FROM member WHERE employeeId = '{crane_employeeId}')"""
await db.query(crane_feedback_graph)


[{'result': [{'body': ['Valeria Crane is a hazard in our team. She has shown no signs of reliability. She is difficult to work with and does not seem motivated. Valeria needs a lot of guidance to complete tasks.',
     'Valeria Crane has shown little in her work to amaze. So far it has been of a level below standard. We have not seen a consistent member of the team. Undertaking tasks has been a poor turn out so far from Valeria.',
     'To simply but, Valeria is currently ranked last in the customer service leaderboard, having missed some basic but important sales target and slacking off at work. She is constantly on her phone to the point where we had to bar the use of phones within the office, causing inconvenience to genuine team members. I am afraid, unless she improves, she may be counting her last few weeks in this company.',
     'Valeria is considered a risk. She has not been performing up to required standards. We have tried working with her to improve and there has not been a

### Explain the difference

So what's going on here? None of these tables are huge (member=75 rows, feedback=225 rows) so I was surprised to see these dramatic of results.

Well, let's use the [EXPLAIN](https://docs.surrealdb.com/docs/surrealql/statements/select/#the-explain-clause) clause.

#### Table Query

You'll see the the table query requires the database to **Iterate Table** over the *entire* **feedback** table because **NO INDEX FOUND**.

Now, we *could* add another column into the feedback table that includes `employeeId` and then make that an index. However, this means that INSERT would be slower because you have to update the index.

Additionally, as we will see, you don't actually need it; you can use a graph query instead!

In [189]:
from json import dumps # formats printing of json

print((crane_feedback_table))
# Explain how the query was preformed
print("Explanation for table lookup by name:")
print(dumps(await db.query(crane_feedback_table + " EXPLAIN"), indent=4))

SELECT rater.name AS rater, body FROM feedback WHERE ratee.employeeId = '65810'
Explanation for table lookup by name:
[
    {
        "result": [
            {
                "detail": {
                    "table": "feedback"
                },
                "operation": "Iterate Table"
            },
            {
                "detail": {
                    "reason": "NO INDEX FOUND"
                },
                "operation": "Fallback"
            }
        ],
        "status": "OK",
        "time": "41.105\u00b5s"
    }
]


#### Graph Query

First, notice that the graph query actually requires two queries.

1. Get the `member` object that has the matching `employeeId` (this is the rightmost query, in parenthesis).
2. Get the information from the **feedback** table.

In a production environment the first query is unlikely to be necessary because you probably already have the member ID from authentication or some other request. But leaving it here is educational.

The first EXPLAIN states that the database uses **Iterate Index** on the **employeeIdIndex** from the **member** table to quickly get the `member` object. This is a good-old-fashioned relational query with an index. It's wicked fast and showcases exactly why we have indexes in the first place.

The second query is the graph query (notice the `->` symbols). That operation is **Iterate Value** on the `member` object.

So, what does this mean?

The query works by tracing edges held in the **gotFeedback** table. By tracing the edges to the appropriate records in the **feedback** table, we neither had to iterate over the table nor build an index to make that iteration faster.

The consequence is that unless an employee has a huge number of feedback entries, it will be much faster to just use our edge table. Even better, we won't experience overhead rebuilding an index on an INSERT to **feedback**.

In [190]:
print(crane_feedback_graph)

print("Explanation for graph lookup:")
# Need two explain statements because there are actually two queries happening
print(dumps(await db.query(crane_feedback_graph.replace(')', ' EXPLAIN) EXPLAIN')), indent=4))

SELECT ->gotFeedback.out.rater.name AS rater, ->gotFeedback.out.body AS body
                            FROM (SELECT id FROM member WHERE employeeId = '65810')
Explanation for graph lookup:
[
    {
        "result": [
            {
                "detail": {
                    "value": {
                        "detail": {
                            "plan": {
                                "index": "employeeIdIndex",
                                "operator": "=",
                                "value": "65810"
                            },
                            "table": "member"
                        },
                        "operation": "Iterate Index"
                    }
                },
                "operation": "Iterate Value"
            }
        ],
        "status": "OK",
        "time": "57.603\u00b5s"
    }
]
